In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
model_id = "meta-llama/Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda:1")
messages = [{"role": "user","content": "test!"}]
empty_magpie_template = tokenizer.apply_chat_template(messages, tokenize=False)
empty_magpie_template

/home/user/repos/open-r1_safety/openr1_v3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.30it/s]


'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\ntest!<|eot_id|>'

In [2]:
empty_magpie_template = '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n'
empty_magpie_inputs = tokenizer(empty_magpie_template, return_tensors="pt").to("cuda:1")
empty_magpie_template
magpie_outputs = model.generate(**empty_magpie_inputs, max_new_tokens=250, do_sample=False, top_p=None, temperature=None)
magpie_decoded = tokenizer.decode(magpie_outputs[0])
print(magpie_decoded)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

The sum of the squares of the first n natural numbers is given by the formula  n(n+1)(2n+1)/6.  What is the sum of the squares of the first 10 natural numbers? 

1^2 + 2^2 + 3^2 + 4^2 + 5^2 + 6^2 + 7^2 + 8^2 + 9^2 + 10^2 =?

To find the answer, use the formula n(n+1)(2n+1)/6.<|eot_id|>


In [3]:
inputs = tokenizer(magpie_decoded, return_tensors="pt").to("cuda:1")
outputs = model.generate(**inputs, max_new_tokens=250, do_sample=False, top_p=None, temperature=None)
decoded = tokenizer.decode(outputs[0])
print(decoded)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

The sum of the squares of the first n natural numbers is given by the formula  n(n+1)(2n+1)/6.  What is the sum of the squares of the first 10 natural numbers? 

1^2 + 2^2 + 3^2 + 4^2 + 5^2 + 6^2 + 7^2 + 8^2 + 9^2 + 10^2 =?

To find the answer, use the formula n(n+1)(2n+1)/6.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

To find the sum of the squares of the first 10 natural numbers, we will use the formula n(n+1)(2n+1)/6 and substitute n = 10.

Sum = n(n+1)(2n+1)/6
Sum = 10(10+1)(2*10+1)/6
Sum = 10(11)(21)/6
Sum = 2310/6
Sum = 385

The sum of the squares of the first 10 natural numbers is 385.<|eot_id|>
